In [4]:
%load_ext autoreload
%autoreload 2
from src.pipeline_real import Pipes
from pathlib import Path
import polars as pl
import yaml

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
pipes = Pipes(
    train_data=Path("data/train.csv"),
    test_data=Path("data/test.csv"),
    params=Path("params.yaml"),
)

In [9]:
train_df = pl.from_pandas(pipes.train_data)
train_df.head()

SEM_NOT,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,CS_SEXO,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,SG_UF,CS_ZONA,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,...,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,MAE_VAC,M_AMAMENTA,ANTIVIRAL,TP_ANTIVIR,HOSPITAL,UTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,AMOSTRA,TP_AMOSTRA,OUT_AMOST,CLASSI_FIN,HISTO_VGM,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FNT_IN_COV,DELTA_UTI,ID
i64,i64,str,str,f64,str,i64,str,i64,i64,str,i64,i64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,i64,f64,f64,f64,f64,f64,f64,...,f64,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,str,i64,i64,str,str,str,f64,f64,f64,f64,f64,str,f64,i64,i64,i64,f64,i64,str
38,37,"""RS""","""005 CRS""",1608.0,"""CAXIAS DO SUL""",430510,"""M""",73,3,"""3073""",6,1,null,"""RS""",2.0,null,2.0,2.0,null,null,null,null,null,null,null,null,null,null,null,1,null,null,null,null,null,null,...,null,null,null,null,1.0,"""ESQUIZOFRENIA""",2.0,null,null,2.0,null,1.0,2.0,2.0,null,null,1.0,1.0,null,4,0,null,null,null,null,null,null,null,null,null,2.0,0,0,0,1.0,-1,"""87360a9a-5516-..."
46,46,"""MS""","""NRS DE PONTA P...",1972.0,"""PONTA PORA""",500660,"""M""",27,3,"""3027""",6,1,4.0,"""MS""",1.0,null,null,null,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,null,null,null,2,null,null,null,null,null,null,...,null,null,null,null,null,null,null,null,null,2.0,null,2.0,null,2.0,1.0,null,1.0,1.0,null,4,0,null,null,null,2.0,2.0,2.0,2.0,null,null,9.0,0,0,0,1.0,-1,"""e5532e89-f7e8-..."
18,17,"""CE""","""1 CRES FORTALE...",1519.0,"""FORTALEZA""",230440,"""F""",2,3,"""3002""",6,4,5.0,"""CE""",1.0,null,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,null,null,2,null,null,null,null,null,null,...,null,null,null,null,null,null,2.0,null,null,2.0,null,1.0,2.0,3.0,9.0,null,1.0,1.0,null,4,0,null,null,null,2.0,2.0,2.0,2.0,6.0,null,1.0,1,1,0,2.0,-1,"""dbaa33c2-d374-..."
4,3,"""SP""","""GVE VII SANTO ...",1332.0,"""MAUA""",352940,"""F""",71,3,"""3071""",5,4,null,"""SP""",1.0,null,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,null,2.0,1,1.0,2.0,2.0,2.0,2.0,1.0,...,2.0,2.0,2.0,null,2.0,null,2.0,null,null,2.0,null,1.0,2.0,2.0,2.0,null,1.0,1.0,null,4,0,null,null,null,2.0,1.0,2.0,2.0,6.0,null,1.0,1,1,0,2.0,-1,"""ecbf8128-42a7-..."
12,11,"""MG""","""JANUARIA""",1470.0,"""SAO JOAO DA PO...",316240,"""F""",91,3,"""3091""",5,4,0.0,"""MG""",2.0,null,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,"""FRAQUEZA""",null,2,null,null,null,null,null,null,...,null,null,null,null,null,null,1.0,null,null,null,null,1.0,2.0,2.0,1.0,null,1.0,1.0,null,4,0,"""621005""","""TRABALHADOR AG...",null,2.0,2.0,2.0,2.0,null,null,2.0,0,0,0,1.0,-1,"""4b74f720-bd35-..."


In [7]:
# categorical data
with open("params.yaml", "r") as f:
    params = yaml.safe_load(f)

categorical_features = params["categorical_features"]
categorical_features

# numerical data
numerical_features = params["numerical_features"]
numerical_features

['SEM_NOT', 'SEM_PRI', 'NU_IDADE_N']

In [14]:
# put the data on correct data type
train_categorical = train_df.select(categorical_features)
train_numerical = train_df.select(numerical_features)

train_categorical = train_categorical.with_columns(pl.all().cast(pl.Utf8).cast(pl.Categorical))
train_numerical = train_numerical.with_columns(pl.all().cast(pl.Int32))

In [22]:
for column in categorical_features:
    print(column, train_categorical.select(pl.col(column).unique()).shape)

SG_UF_NOT (27, 1)
CO_REGIONA (311, 1)
CO_MUN_NOT (2940, 1)
CS_SEXO (3, 1)
COD_IDADE (176, 1)
CS_GESTANT (8, 1)
CS_RACA (6, 1)
CS_ESCOL_N (7, 1)
SG_UF (27, 1)
CS_ZONA (4, 1)
SURTO_SG (3, 1)
NOSOCOMIAL (3, 1)
AVE_SUINO (4, 1)
FEBRE (3, 1)
TOSSE (3, 1)
GARGANTA (3, 1)
DISPNEIA (3, 1)
DESC_RESP (3, 1)
SATURACAO (3, 1)
DIARREIA (3, 1)
VOMITO (3, 1)
OUTRO_SIN (3, 1)
PUERPERA (3, 1)
FATOR_RISC (2, 1)
CARDIOPATI (3, 1)
HEMATOLOGI (3, 1)
SIND_DOWN (3, 1)
HEPATICA (3, 1)
ASMA (3, 1)
DIABETES (3, 1)
NEUROLOGIC (3, 1)
PNEUMOPATI (3, 1)
IMUNODEPRE (3, 1)
RENAL (3, 1)
OBESIDADE (3, 1)
OUT_MORBI (3, 1)
VACINA (3, 1)
ANTIVIRAL (3, 1)
HOSPITAL (3, 1)
UTI (3, 1)
SUPORT_VEN (4, 1)
RAIOX_RES (7, 1)
RAIOX_OUT (11261, 1)
AMOSTRA (3, 1)
TP_AMOSTRA (6, 1)
HISTO_VGM (4, 1)
DOR_ABD (3, 1)
FADIGA (3, 1)
PERD_OLFT (3, 1)
PERD_PALA (3, 1)
TOMO_RES (7, 1)
VACINA_COV (3, 1)
DOSE_1_COV (2, 1)
DOSE_2_COV (2, 1)
FNT_IN_COV (2, 1)


In [25]:
train_categorical.select(pl.col("CO_MUN_NOT").value_counts())

CO_MUN_NOT
struct[2]
"{""430060"",895}"
"{""311230"",222}"
"{""231335"",19}"
"{""312950"",50}"
"{""430510"",5871}"
"{""231380"",11}"
"{""510600"",11}"
"{""310090"",168}"
"{""292590"",2}"


In [29]:
df = train_df.select(pl.col(["CO_MUN_NOT", "CO_REGIONA"])).to_pandas()

In [36]:
from feature_engine.encoding import RareLabelEncoder

rare_encoder = RareLabelEncoder(tol=0.01, n_categories=10)
rare_encoder.fit_transform(df)

ValueError: Some of the variables to transform contain NaN. Check and remove those before using this transformer.

In [34]:
# convert a dataframe pandas columns to categorical
df = df.astype({"CO_MUN_NOT": "category", "CO_REGIONA": "category"})
df

,CO_MUN_NOT,CO_REGIONA
0,430510,1608.0
1,500660,1972.0
2,230440,1519.0
3,352940,1332.0
4,316240,1470.0
...,...,...
1300975,411370,1371.0
1300976,355030,1331.0
1300977,330455,NaN
1300978,240810,1416.0


In [39]:
train_categorical.shape

(1300980, 55)

In [42]:
train_numerical.describe()

describe,SEM_NOT,SEM_PRI,NU_IDADE_N
str,f64,f64,f64
"""count""",1.30098e6,1.30098e6,1.30098e6
"""null_count""",0.0,0.0,0.0
"""mean""",20.190366,19.395701,52.451015
"""std""",12.866569,13.204212,25.22482
"""min""",1.0,1.0,-9.0
"""max""",52.0,52.0,150.0
"""median""",18.0,17.0,56.0
